In [1]:
import numpy as np

In [4]:
import pandas as pd

In [10]:
test_csv = pd.read_csv("data/test.csv")
train_csv = pd.read_csv("data/train.csv")

In [11]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
'''
칼럼 정의

survival : 생존여부 (0 : 사망, 1 : 생존)
Pclass : 티켓 클래스 ( 1 : 1등급, 2 : 2등급, 3 : 3등급)
sex : 성별 (male, female) -> 추후 0,1
SibSp : 타이타닉호에 탑승한 형제/배우자 (0,1,2,3,4,5,8)
Parch : 타이타닉호에 탑승한 부모/ 자녀 수
ticket : 티켓 번호
fare : 행인?
Cabin : 객실번호
Embarked : 승선항 ( C = Cherbourg, Q = Queenstown, S = Southampton)
'''

In [16]:
train_csv['Sex']

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object